In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import numpy as np
from numpy import arange
from numpy.random import default_rng
from numpy.typing._extended_precision import float128
from scipy.optimize import minimize, Bounds, optimize
from scipy.optimize import brute
from skopt import gp_minimize
import skopt.plots as skplt

/Users/lucaswebb/.pyenv/versions/3.9.6/envs/ukraine/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ModuleNotFoundError: No module named 'numpy.typing._extended_precision'

In [ ]:
# Read in data and clean it

data2013 = pd.read_csv('2013_Events.csv')
data2012 = pd.read_csv('2012_Events.csv')
data2011 = pd.read_csv('2011_Events.csv')
data20092010 = pd.read_csv('2009_2010_Events.csv')

data2013 = data2013[['Event start date', 'Number of protesters', 'Location']]
data2012 = data2012[['Event start date', 'Number of protesters', 'Location']]
data2011 = data2011[['Event start date', 'Number of protesters', 'Location']]
data20092010 = data20092010[['Event date', '# protester', 'Location']]
data20092010.columns = ['Event start date', 'Number of protesters', 'Location']

# Combine multiple years into one dataframe
data = pd.concat([data2013, data2012, data2011, data20092010])

# Sort by event date (descending)
data['Event start date'] = pd.to_datetime(data['Event start date'])
data = data.sort_values(by = 'Event start date')

data_kyiv = data.loc[data['Location'] == 'Kyiv']

data['Number of events'] = 0
data['Number of protesters'] = pd.to_numeric(data['Number of protesters'], errors='coerce')

data_kyiv['Number of events'] = 0
data_kyiv['Number of protesters'] = pd.to_numeric(data_kyiv['Number of protesters'], errors='coerce')

# Combine entries that occur on the same date and sum the number of protesters
data = data.groupby(data['Event start date']).aggregate({'Number of events': 'size', 'Number of protesters': 'sum'})
data_kyiv = data_kyiv.groupby(data_kyiv['Event start date']).aggregate({'Number of events': 'size', 'Number of protesters': 'sum'})

In [ ]:
data.tail()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(data_kyiv['Number of events'].rolling(10).mean(), label='data')

In [ ]:
# Define function to calculate background activity level

def calculateN0(df, sigma, deltaT):
    return df['Number of events'].rolling(window=deltaT, win_type="gaussian", center=True).mean(std=sigma)
    # N0 is Null for the first and last deltaT/2 values

In [ ]:
# Plot data and N0
plt.figure(figsize=(20,10))
plt.plot(data['Number of events'].rolling(10).mean(), label='data')
plt.plot(calculateN0(data, 360, 120), label='baseline')

In [ ]:
print(MSE(data['Number of events'], calculateN0(data, 20, 10)))

In [ ]:
# Define a function to create a new dataframe to be used in simulation

def createSimulationDF(df, sigma, deltaT):
    data1 = df.copy()
    data1['N0'] = calculateN0(data1, sigma, deltaT)
    data1['simulated'] = 0
    data1.reset_index(inplace=True)
    data1['Event start date'] = pd.to_numeric(pd.to_datetime(data1['Event start date']))
    data1 = data1[int(deltaT/2):int(-deltaT/2-1)]
    data1['simulated'][int(deltaT/2)] = 1
    return data1

In [ ]:
# Define helper function to create simulated data

def simulaterHelper(x, args):
    arr = x[-1]
    d = (-x[:-1,0]/86400 + arr[0]/86400)/1000000000
    psum = x[:-1,1] * (np.exp((-(d-1))/args[0][1]) - np.exp(-d/args[0][1]))
    # arr[4] = np.random.poisson(arr[3] + args[0][0]*psum.sum(), 1)[0]
    # if arr[4] == 0:
    #     arr[4] = 1
    arr[4] = arr[3] + args[0][0]*psum.sum()
    # arr[4] = psum.sum()
    # arr[4] = d.sum()
    return arr

In [ ]:
# Define function to create simulated data given simulationDF (to be used when fitting Nsec and Texcite)

def simulate(df, Nsec, Texcite):
    simulated_data = df.expanding(method="table").apply(simulaterHelper, Nsec, Texcite, raw=True, engine="numba")
    # simulated_data['Event start date'] = pd.to_datetime(simulated_data['Event start date'])
    return simulated_data

In [ ]:
simulationDF = createSimulationDF(data, 360, 120)

In [ ]:
simulated = simulate(simulationDF, 0.1, 0.1)

In [ ]:
simulated.head()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulated['simulated'].rolling(10).mean(), label='model')

In [ ]:
# Loss and error functions
def computeLoss(model, actual):
    return (model - (actual * np.log(model))).sum()

def MSE(model, actual):
    return np.power((actual - model).sum(), 2).sum() / len(actual)

In [ ]:
print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(MSE(simulated['simulated'], simulated['Number of events']))


In [ ]:
def functionToMinimize(x):
    sim = simulate(simulationDF, x[0], x[1])
    return computeLoss(sim['simulated'].astype(np.longdouble), sim['Number of events'].astype(np.longdouble))

In [ ]:
x0 = [0.174, 9.9]
# res = minimize(functionToMinimize, x0, method = 'Nelder-Mead', options={'maxiter': 10})
res = brute(functionToMinimize, (slice(0.01, 0.9, 0.01), slice(0.1, 10, 1)), finish=None)#, full_output=True)
# , options={'maxiter': 1000}
# res = minimize(functionToMinimize, x0)
print(res)

In [ ]:
# Compare baseline with baseline + contagion

print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
# Lets try fitting a 3 parameter model with a constant baseline background

pd.options.mode.chained_assignment = None  # default='warn'
def createSimulationDF2(df, n):
    data1 = df.copy()
    data1['N0'] = n
    data1['simulated'] = 0
    data1.reset_index(inplace=True)
    data1['Event start date'] = pd.to_numeric(pd.to_datetime(data1['Event start date']))
    data1['simulated'][0] = 1
    return data1

def functionToMinimize2(x):
    sim = simulate(createSimulationDF2(data, x[2]), x[0], x[1])
    return computeLoss(sim['simulated'].astype(np.longdouble), sim['Number of events'].astype(np.longdouble))

In [ ]:
x0 = [0.174, 9.9, 10]
res = minimize(functionToMinimize2, x0, method = 'Nelder-Mead', options={'maxiter': 10})
# , options={'maxiter': 1000}
# res = minimize(functionToMinimize, x0)
print(res)

In [ ]:
# Lets just look at Euromaiden (started November 21st 2013)

data_2013 = data.loc['2013-11-16':]

In [ ]:
# Find average number of events per day up until Euromaiden
data.loc[:'2013-11-01']['Number of events'].mean()
# 11.56694560669456

In [ ]:
# Look at constant background model
data_2013N0 = createSimulationDF2(data_2013, 3.25)

# Plot data and N0
plt.figure(figsize=(20,10))
plt.plot(data_2013N0['Number of events'], label='data')
plt.plot(data_2013N0['N0'], label='background')

In [ ]:
data_2013N0.shape[0]

In [ ]:
simulationDF = data_2013N0

In [ ]:
%%timeit
sim = simulate(simulationDF, 0.63, 1.76)

In [ ]:
simulationDF = data_2013N0
x0 = [0.76, 1.1]
# res = minimize(functionToMinimize, x0, method = 'Nelder-Mead', options={'maxiter': 10})
res = brute(functionToMinimize, (slice(0.01, 0.9, 0.01), slice(0.1, 15, 1)), finish=minimize)
print(res)
# 0.71725041, 1.22031264 after 100000 iterations

In [ ]:
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'], label='data')
plt.plot(simulated['simulated'], label='model')

print(computeLoss(simulated['Number of events'], simulated['Number of events']))
print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
# Apply weighted average background model to Euromaiden

data_2013N0 = createSimulationDF(data_2013, 300, 20)

# Plot data and N0
plt.figure(figsize=(20,10))
plt.plot(data_2013N0['Number of events'], label='data')
plt.plot(data_2013N0['N0'], label='background')

In [ ]:
data_2013N0.head()

In [ ]:
simulationDF = data_2013N0
x0 = [0.76, 7]
# res = minimize(functionToMinimize, x0, method = 'Nelder-Mead', options={'maxiter': 20000})
res = brute(functionToMinimize, (slice(0.01, 0.9, 0.01), slice(0.1, 5, 0.1)), finish=None)#, full_output=True)
print(res)

In [ ]:
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'], label='data')
plt.plot(simulated['simulated'], label='model')

print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
simulated2 = simulate(simulationDF, 0.06, 0.1)

plt.figure(figsize=(20,10))
plt.plot(simulated2['Number of events'], label='data')
plt.plot(simulated2['simulated'], label='model')

In [ ]:
# Look at entire dataset with baseline
simulationDF = createSimulationDF2(data, 12)

plt.figure(figsize=(20,10))
plt.plot(simulationDF['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulationDF['N0'], label='baseline')

In [ ]:
res = brute(functionToMinimize, (slice(0.01, 0.4, 0.01), slice(0.1, 10, 0.1)), finish=None)#, full_output=True)
print(res)

In [ ]:
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulated['simulated'].rolling(10).mean(), label='model')

In [ ]:
print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
# Look just at Kyiv

kyiv_N0 = createSimulationDF(data_kyiv, 300, 100)

# Plot data and N0
plt.figure(figsize=(20,10))
plt.plot(kyiv_N0['Number of events'].rolling(10).mean(), label='data')
plt.plot(kyiv_N0['N0'], label='background')

In [ ]:
simulationDF = kyiv_N0
res = brute(functionToMinimize, (slice(0.01, 0.9, 0.01), slice(0.1, 15, 1)), finish=None)#, full_output=True)
print(res)

In [ ]:
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot((simulated['simulated']).rolling(10).mean(), label='model')

print(computeLoss(simulated['Number of events'], simulated['Number of events']))
print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))


In [ ]:
simulated.tail()

In [ ]:
data_kyiv['Number of events'].mean()


In [ ]:
# Kyiv with constant background

kyiv_N0 = createSimulationDF2(data_kyiv, 3.25)

# Plot data and N0
plt.figure(figsize=(20,10))
plt.plot(kyiv_N0['Number of events'].rolling(10).mean(), label='data')
plt.plot(kyiv_N0['N0'], label='background')

In [ ]:
simulationDF = kyiv_N0
res = brute(functionToMinimize, (slice(0.01, 0.5, 0.01), slice(0.1, 5, 0.1)), finish=None)#, full_output=True)
print(res)
#[0.19325032 2.09839541]

In [ ]:
simulated = simulate(simulationDF, 0.19, 2.1)

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulated['simulated'].rolling(10).mean(), label='model')

print(computeLoss(simulated['Number of events'], simulated['Number of events']))
print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
simulated.head()

In [ ]:
# Test the predictive power of constant baseline in kyiv
# Train on the first half
kyiv_N0 = createSimulationDF2(data_kyiv.loc[:'2012-01-01'], data_kyiv.loc[:'2012-01-01']['Number of events'].mean())
simulationDF = kyiv_N0
res = brute(functionToMinimize, (slice(0.01, 0.5, 0.01), slice(0.1, 5, 0.1)), finish=None)#, full_output=True)
print(res)

In [ ]:
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulated['simulated'].rolling(10).mean(), label='model')

print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
# Test on the second half
kyiv_N0 = createSimulationDF2(data_kyiv.loc['2012-01-01':], data_kyiv.loc['2012-01-01':]['Number of events'].mean())
simulationDF = kyiv_N0
simulated = simulate(simulationDF, res[0], res[1])

plt.figure(figsize=(20,10))
plt.plot(simulated['Number of events'].rolling(10).mean(), label='data')
plt.plot(simulated['simulated'].rolling(10).mean(), label='model')

print(computeLoss(simulated['simulated'], simulated['Number of events']))
print(computeLoss(simulated['N0'], simulated['Number of events']))

In [ ]:
# Look at just euromaiden with constant background to compare with distance model and verify models produce same output

toCompare = data_kyiv.loc['2013-11-16':]

index = pd.date_range('2013-11-16','2013-12-31')
toCompare = toCompare.reindex(index, fill_value=0.0)
toCompare = toCompare.reset_index()
del toCompare['index']

def l2Error(simulated, actual):
    return np.power(actual - simulated, 2).sum()

def functionToMinimize3(params):
    sim = simulate(simulationDF, params[0], params[1])
    return l2Error(sim['simulated'], toCompare['Number of events'])

In [ ]:
data_2013N0 = createSimulationDF2(data_2013, 3.25)
simulationDF = data_2013N0

In [ ]:
functionToMinimize3([0.1, 1])

In [ ]:
sim = simulate(simulationDF, 0.1, 1)
plt.figure(figsize=(20,10))
plt.plot(sim['simulated'], label='model')
plt.plot(sim['N0'], label='background')
plt.plot(toCompare['Number of events'], label='data')
plt.legend()

In [ ]:
sim.head()


In [ ]:
%%time
res = gp_minimize(functionToMinimize3, [(0.0, 1.0), (0.0001, 20.0)], n_calls=25, noise=1e-10, n_initial_points=1, random_state=123)

In [ ]:
res.x

In [ ]:
res.fun

In [ ]:
sim = simulate(simulationDF, res.x[0], res.x[1])
plt.figure(figsize=(20,10))
plt.plot(sim['simulated'], label='model')
plt.plot(sim['N0'], label='background')
plt.plot(toCompare['Number of events'], label='data')
plt.legend()

In [ ]:
skplt.plot_convergence(res);
skplt.plot_objective(res);
skplt.plot_evaluations(res);